In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

np.random.seed(0)
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_train.csv",index_col = 0)

def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative (x):
    return sigmoid(x)*(1-sigmoid(x))

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z),axis = 1,keepdims = True)

def softmax_derivative(x):
    return softmax(x)*(1 - softmax(x))

def R2(out,y):
    return 1 - (np.sum((out-y)**2))/(np.sum((y - np.mean(y))**2))

def Accuracy(y_pred , y_test):
    
    count = 0
    y_pred = np.argmax(y_pred,axis = 1)
    y_test = np.argmax(y_test,axis = 1)
    for i in range(y_pred.size):
        if (y_pred[i] - y_test[i] == 0):
            count+=1
    return count/y_test.size

class Layer():
    def __init__(self,n_inputs,n_neurons):
        self.weights = np.random.randn(n_inputs,n_neurons)
        self.bias = np.random.randn(1,n_neurons)
    def forward(self,inputs):
        self.outputs = inputs @ self.weights + self.bias
        
    def backward(self,error,weights):
        self.error = ( error @ np.transpose(weights) ) * sigmoid_derivative(self.outputs)

    def set_error(self,error):
        self.error = error

    def fill():
        pass
 
def forward_propagate(x,layers,act):
    
    n = len(layers)-1
    for i in range(n):
        
        if i == 0 :
            layers[0].forward(x)
            act[0] = sigmoid(layers[0].outputs)
        else:
            layers[i].forward(act[i-1])
            act[i] = sigmoid(layers[i].outputs)

    layers[n].forward(act[n-1])
    act[n] = softmax(layers[n].outputs)

    return layers,act

def backward_propagate(x,y,learning_rate,layers,act):
    size = act[-1].shape[0]
    n = len(layers) -1
    error = -(act[-1]-y)

    for i in range(n,0,-1):
        if i == (n):
            error *= softmax_derivative(layers[-1].outputs)
            layers[i].set_error(error)
            Jw = np.array(act[i-1]).T @ layers[i].error 
            Jb = np.sum(error,axis = 0)

        else :
            layers[i].backward(layers[i+1].error,layers[i+1].weights)
            Jw = np.array(act[i-1]).T @ layers[i].error
            Jb = np.sum(layers[i].error,axis = 0 )
        layers[i].weights += learning_rate/size*Jw
        layers[i].bias += learning_rate/size*Jb

    layers[0].backward(layers[1].error,layers[1].weights)
    Jw = np.array(x).T @ layers[0].error
    Jb = np.sum(layers[0].error,axis = 0)

    layers[0].weights += learning_rate/size*Jw
    layers[0].bias += learning_rate/size*Jb

    return layers

In [79]:
# Analysis of Labelled Data

for i in range(10):
    xi = df_train.iloc[i,1:].to_numpy()
    xi = xi.reshape([28,28])
    img = np.uint8(xi)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2_imshow(img)

classes = pd.DataFrame([np.unique(df_train.label.values),np.bincount(df_train.label.values)],index = ['labels','Number of labels'])
print(classes)

train_data = df_train.iloc[5000:,1:].values
test_data = df_train.iloc[:5000,1:].values
label_train = df_train.iloc[5000:,0].to_numpy()
label_test = df_train.iloc[:5000,0].to_numpy()

x = (train_data - np.mean(train_data))/255
x_test = (test_data - np.mean(train_data))/255

row , col = x.shape
y = np.zeros([row,10])
y_test = np.zeros([label_test.shape[0],10])

for i in range(row):
    y[i , label_train[i]] = 1 
    if i < label_test.shape[0] :
        y_test[i , label_test[i]] = 1
# y = y.reshape([label_train.shape[0],1])

n = 2  #2
m = 20   #20
layers , act = [1]*(n+1),[1]*(n+1)

for i in range(n):
    if i == 0 :
        layers[0] = Layer(784,m)
        layers[0].forward(x)
        act[0] = sigmoid(layers[0].outputs)
    else:
        layers[i] = Layer(m,m)
        layers[i].forward(act[i-1])
        act[i] = sigmoid(layers[i].outputs)
layers[n] = Layer(m,10)
layers[n].forward(act[n-1])
act[n] = softmax(layers[n].outputs)

layers[0].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw1.npy")
layers[0].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb1.npy")
layers[1].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw2.npy")
layers[1].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb2.npy")
layers[2].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw3.npy")
layers[2].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb3.npy")

                     0     1     2     3     4     5     6     7     8     9
labels               0     1     2     3     4     5     6     7     8     9
Number of labels  3000  3000  3000  3000  3000  3000  3000  3000  3000  3000


In [80]:
learning_rate=0.1
n_iterations = 100000
start = 0
end = x.shape[0]
step = x.shape[0]//10

for i in range(n_iterations):
    for k in range(start,end,step):

        layers,act = forward_propagate(x[k:k+step,:],layers,act)
        layers = backward_propagate(x[k:k+step,:],y[k:k+step,:],learning_rate,layers,act)
        layers,act = forward_propagate(x,layers,act)
        loss = -np.mean(y*np.log(act[-1]))

        if (i % 10 == 0):

            print("loss = ",loss)
            layers_test , act_test = forward_propagate(x_test,layers,act)
            score = Accuracy(np.round(act_test[-1]),y_test)
            print("Accuracy = ",score*100,"%")
            print("Y_test = ",np.argmax(y_test[960:1000,:],axis = 1))
            print("Y_pred = ",np.argmax(act_test[-1][960:1000,:],axis = 1))


loss =  0.037714109162410386
Accuracy =  80.30000000000001 %
Y_test =  [3 5 5 6 1 2 9 4 6 3 1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1
 7 1 2]
Y_pred =  [3 7 5 6 1 6 9 4 6 6 1 7 6 4 1 0 4 6 4 3 1 6 4 5 5 1 9 1 5 4 6 5 6 6 1 1 1
 7 1 2]
loss =  0.0377142438611292
Accuracy =  80.30000000000001 %
Y_test =  [3 5 5 6 1 2 9 4 6 3 1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1
 7 1 2]
Y_pred =  [3 7 5 6 1 6 9 4 6 6 1 7 6 4 1 0 4 6 4 3 1 6 4 5 5 1 9 1 5 4 6 5 6 6 1 1 1
 7 1 2]
loss =  0.037712858109424555
Accuracy =  80.28 %
Y_test =  [3 5 5 6 1 2 9 4 6 3 1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1
 7 1 2]
Y_pred =  [3 7 5 6 1 6 9 4 6 6 1 7 6 4 1 0 4 6 4 3 1 6 4 5 5 1 9 1 5 4 6 5 6 6 1 1 1
 7 1 2]
loss =  0.03771016773044155
Accuracy =  80.28 %
Y_test =  [3 5 5 6 1 2 9 4 6 3 1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1
 7 1 2]
Y_pred =  [3 7 5 6 1 6 9 4 6 6 1 7 6 4 1 0 4 6 4 3 1 6 4 5 5 1 9 1 5 4 6 5 6 6 1 1 1
 7 1 2]
loss =  0.03770839383765515
Accuracy = 

KeyboardInterrupt: ignored

In [81]:
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw1.npy",layers[0].weights)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb1.npy",layers[0].bias)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw2.npy",layers[1].weights)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb2.npy",layers[1].bias)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw3.npy",layers[2].weights)
np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb3.npy",layers[2].bias)

In [82]:
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_test.csv")
X_test = df_test.iloc[:,1:].values
ids = df_test.iloc[:,0].values

Test_data = (X_test - np.mean(train_data))/255
layers_test,act_test = forward_propagate(Test_data,layers,act)
Y_pred = np.argmax(layers_test[-1].outputs,axis = 1)

res = pd.DataFrame([ids,Y_pred],index = ["Ids","Prediction"]).T
print(res)
res.to_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nnc_result.csv")

        Ids  Prediction
0     25672           0
1     59964           1
2     66230           2
3     50801           2
4     83307           4
...     ...         ...
9995  71009           0
9996  67472           6
9997  99226           8
9998  58519           2
9999  80458           2

[10000 rows x 2 columns]
